Here we will learn how to change the Parameterization such that you can set the value of constrained Parameters (by having the constraint solve for a different Parameter instead).

# Setup

In [1]:
import phoebe
from phoebe import u,c

In [2]:
logger = phoebe.logger(clevel='WARNING')

In [3]:
b = phoebe.default_binary()

# Advanced Constraints

In the default binary, there are a significant number of constrained Parameters.

In [5]:
print(b.filter(context='constraint').qualifiers)

['ecosw', 'mean_anom', 'asini', 't0_ref', 'period', 'long_an', 'mass', 't0_perpass', 'esinw', 'freq', 'incl', 'requiv_max', 'sma', 'irrad_frac_lost_bol']


Let's look at mass, which is *constrained* by default according to Kepler's third law.

In [6]:
print(b.get_parameter('mass', component='primary', context='component'))

Parameter: mass@primary@component
                       Qualifier: mass
                     Description: Mass
                           Value: 0.9988131358 solMass
                  Constrained by: sma@binary@component, period@binary@component, q@binary@component
                      Constrains: None
                      Related to: sma@binary@component, period@binary@component, q@binary@component



In [7]:
print(b.get_parameter('mass', component='primary', context='constraint'))

Constrains (qualifier): mass
Expression in solar units (value): (39.478418 * ({sma@binary@component} ** 3.000000)) / ((({period@binary@component} ** 2.000000) * (1.000000 + {q@binary@component})) * 2942.206217504418873431859537959099)
Current Result (result): 0.998813135806 solMass


Here we see the 4 parameters that are involved in Kepler's third law.  PHOEBE allows you to freely set 3 of these 4 (sma, period, q) and automatically uses these values to compute that mass.

However, let's say that you wanted to set the mass (perhaps you know the mass, but don't know the semi-major axis as well).  This can be done via the 'flip_constraint' method.  The easiest way to use this correctly is to make sure our keywords return the correct Constraint Parameter via 'get_constraint' and then use 'flip_constraint'.

In [8]:
b.get_constraint(qualifier='mass', component='primary')

<ConstraintParameter: {mass@primary@component} = (39.478418 * ({sma@binary@component} ** 3.000000)) / ((({period@binary@component} ** 2.000000) * (1.000000 + {q@binary@component})) * 2942.206217504418873431859537959099) (solar units) => 0.998813135806 solMass>

Now we just add `solve_for='sma'` to "flip" this constraint to solve for 'sma' instead of 'mass'.

In [9]:
b.flip_constraint(qualifier='mass', component='primary', solve_for='sma')

<ConstraintParameter: {sma@binary@component} = (((({mass@primary@component} * ({period@binary@component} ** 2.000000)) * (1.000000 + {q@binary@component})) * 2942.206217504418873431859537959099) / 39.478418) ** (1./3) (solar units) => 5.29999999999 solRad>

Now we're allowed to set the mass and we'll see that the value of sma is automatically computed.

In [10]:
b.set_value('mass', component='primary', value=1.2)

In [11]:
b.get_value('sma', component='binary', context='component')

5.6343203568093525

# Exercise

Flipping constraints could be particularly useful if you have an observational constraint on 'asini' (say from the amplitude of RVs) and want to leave asini fixed as you fit for the inclination.  Flip the constraint so it is possible to adjust the values of both 'asini' and 'incl'.  (**NOTE** you may want to either start fresh or re-flip the Kepler's third law constraint back to solve for mass first).

Now that you can change the value of 'asini', set it to 20 (solar radii... we'll talk about units in the next tutorial), adjust the inclination, and show that 'sma' is adjusting automatically to conserve 'asini'.